In [1]:
import chromadb

In [2]:
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.create_collection("documents")

In [40]:
import polars as pl

quotes = pl.read_csv("./quotes.csv")
quotes.head()

,quote,author,tags
i64,str,str,str
0,"""“Be yourself; everyone else is…","""Oscar Wilde""","""['be-yourself', 'gilbert-perre…"
1,"""“I'm selfish, impatient and a …","""Marilyn Monroe""","""['best', 'life', 'love', 'mist…"
2,"""“Two things are infinite: the …","""Albert Einstein""","""['human-nature', 'humor', 'inf…"
3,"""“So many books, so little time…","""Frank Zappa""","""['books', 'humor']"""
4,"""“A room without books is like …","""Marcus Tullius Cicero""","""['books', 'simile', 'soul']"""


In [46]:
quotes = quotes.rename({'':'index'})

In [4]:
quotes.shape

(2508, 4)

In [5]:
KEY = "YOUR API KEY"

In [6]:
import chromadb.utils.embedding_functions as embedding_functions
google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=KEY)


In [47]:
N = 12
quotes = quotes[:N]
quotes.head()

index,quote,author,tags
i64,str,str,str
0,"""“Be yourself; everyone else is…","""Oscar Wilde""","""['be-yourself', 'gilbert-perre…"
1,"""“I'm selfish, impatient and a …","""Marilyn Monroe""","""['best', 'life', 'love', 'mist…"
2,"""“Two things are infinite: the …","""Albert Einstein""","""['human-nature', 'humor', 'inf…"
3,"""“So many books, so little time…","""Frank Zappa""","""['books', 'humor']"""
4,"""“A room without books is like …","""Marcus Tullius Cicero""","""['books', 'simile', 'soul']"""


In [21]:
from pprint import pprint

In [22]:
l = []

for i in range(10):
    l.append(quotes['quote'][i])

pprint(l)

['“Be yourself; everyone else is already taken.”',
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of "
 "control and at times hard to handle. But if you can't handle me at my worst, "
 "then you sure as hell don't deserve me at my best.”",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure "
 'about the universe.”',
 '“So many books, so little time.”',
 '“A room without books is like a body without a soul.”',
 "“Be who you are and say what you feel, because those who mind don't matter, "
 "and those who matter don't mind.”",
 "“You've gotta dance like there's nobody watching,Love like you'll never be "
 "hurt,Sing like there's nobody listening,And live like it's heaven on earth.”",
 "“You know you're in love when you can't fall asleep because reality is "
 'finally better than your dreams.”',
 '“You only live once, but if you do it right, once is enough.”',
 '“Be the change that you wish to see in the world.”']


In [29]:
quotes_list = quotes['quote'][1:].to_list()
vectors = google_ef(quotes_list)
len(vectors)

11

In [48]:
ids = [f"id{x}" for x in quotes['index'][1:].to_list()]
ids

['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9', 'id10', 'id11']

In [12]:
collection = chroma_client.get_or_create_collection(name="quotes")

In [49]:
collection.add(
    documents=quotes_list,
    ids=ids,
    embeddings=vectors
)

Add of existing embedding ID: id1
Insert of existing embedding ID: id1


In [50]:
collection.count()

11

In [55]:
query = "you only live once"
query_embeddings = google_ef([query])

collection.query(
    query_embeddings = query_embeddings,
    n_results = 3
)

{'ids': [['id8', 'id10', 'id1']],
 'distances': [[0.1451733261346817, 0.362739622592926, 0.38663768768310547]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['“You only live once, but if you do it right, once is enough.”',
   "“In three words I can sum up everything I've learned about life: it goes on.”",
   '“Be yourself; everyone else is already taken.”']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}